<a href="https://colab.research.google.com/github/muxywj/.github/blob/main/%EC%9D%BC%EB%AA%B0%EC%9D%BC%EC%B6%9Capi%20%EC%97%91%EC%85%80%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API 불러오기 테스트

In [33]:
import requests

serviceKey = 'Pm/kNFEx4ruhUTK4w5WFeS3weybFEum+p40tGIcQ6GkNQDV7M+SAaRTwqg5Xnt+CSdly/BCSoFl5wJgc86+gRg=='
locdate = '20230101'
location = '서울'
dataType = 'xml'

url = 'https://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getAreaRiseSetInfo'
params = {'serviceKey' : serviceKey, 'locdate': locdate, 'location': location, 'dataType' : dataType }

response = requests.get(url, params=params)
print(response.text)

Unauthorized



# 데이터 불러오기

## API 호출
2026년 1월 1일부터 12월 31일까지의 모든 날짜를 생성.


In [20]:
from datetime import date, timedelta

# Define start and end dates for 2026
start_date = date(2026, 1, 1)
end_date = date(2026, 12, 31)

# Initialize an empty list to store the dates
dates_2026 = []

# Loop through dates from start_date to end_date
current_date = start_date
while current_date <= end_date:
    # Format the date as 'YYYYMMDD' string and append to the list
    dates_2026.append(current_date.strftime('%Y%m%d'))
    # Move to the next day
    current_date += timedelta(days=1)

# Verify the first few and last few dates
print(f"First 5 dates: {dates_2026[:5]}")
print(f"Last 5 dates: {dates_2026[-5:]}")
print(f"Total number of dates: {len(dates_2026)}")

First 5 dates: ['20260101', '20260102', '20260103', '20260104', '20260105']
Last 5 dates: ['20261227', '20261228', '20261229', '20261230', '20261231']
Total number of dates: 365


## Collect Sunrise/Sunset Data

### Subtask:
생성된 각 날짜에 대해 한국천문연구원_출몰시각 정보 API(getAreaRiseSetInfo)를 호출하고, 응답 XML에서 지역의 일출(sunrise) 및 일몰(sunset) 시각을 추출


In [27]:
import xml.etree.ElementTree as ET
import time

# Initialize an empty list to store the collected data
sunrise_sunset_data = []

# Loop through each date in dates_2026
for current_locdate in dates_2026:
    # Update the locdate parameter for the API call
    params['locdate'] = current_locdate

    try:
        # Make the API call
        response = requests.get(url, params=params)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        # Parse the XML response
        root = ET.fromstring(response.text)

        # Find the item element containing the sunrise and sunset information
        item = root.find('.//item')

        if item is not None:
            sunrise_tag = item.find('sunrise')
            sunset_tag = item.find('sunset')

            sunrise_time = sunrise_tag.text.strip() if sunrise_tag is not None else 'N/A'
            sunset_time = sunset_tag.text.strip() if sunset_tag is not None else 'N/A'

            # Append the extracted data to the list
            sunrise_sunset_data.append({
                'date': current_locdate,
                'sunrise': sunrise_time,
                'sunset': sunset_time
            })
        else:
            print(f"Warning: No item found for date {current_locdate}")
            sunrise_sunset_data.append({
                'date': current_locdate,
                'sunrise': 'N/A',
                'sunset': 'N/A'
            })

    except requests.exceptions.RequestException as e:
        print(f"Error making API call for {current_locdate}: {e}")
        sunrise_sunset_data.append({
            'date': current_locdate,
            'sunrise': 'API_ERROR',
            'sunset': 'API_ERROR'
        })
    except ET.ParseError as e:
        print(f"Error parsing XML for {current_locdate}: {e}")
        sunrise_sunset_data.append({
            'date': current_locdate,
            'sunrise': 'XML_ERROR',
            'sunset': 'XML_ERROR'
        })

    # Add a short delay to avoid overwhelming the API server
    time.sleep(0.1)

# Print the first few entries to verify the collected data
print("Collected Sunrise/Sunset Data (first 5 entries):")
for entry in sunrise_sunset_data[:5]:
    print(entry)

print("Collected Sunrise/Sunset Data (last 5 entries):")
for entry in sunrise_sunset_data[-5:]:
    print(entry)

print(f"Total entries collected: {len(sunrise_sunset_data)}")

KeyboardInterrupt: 

## Data 프레임 구성
수집된 날짜, 일출 시각, 일몰 시각 데이터를 포함하는 pandas DataFrame을 생성. DataFrame의 컬럼은 '날짜', '일출 시각', '일몰 시각'으로 구성


In [21]:
import pandas as pd

# Create a pandas DataFrame from the collected data
df_sunrise_sunset = pd.DataFrame(sunrise_sunset_data)

# Rename the columns to the specified Korean names
df_sunrise_sunset.columns = ['날짜', '일출 시각', '일몰 시각']

# Display the first 5 rows of the DataFrame to verify
print(df_sunrise_sunset.head())

         날짜 일출 시각 일몰 시각
0  20260101  0747  1724
1  20260102  0747  1725
2  20260103  0747  1726
3  20260104  0747  1727
4  20260105  0747  1728


##생성된 DataFrame을 엑셀로 저장

In [31]:
output_filename = 'sunrise_sunset_2026.xlsx'
df_sunrise_sunset.to_excel(output_filename, index=False)
print(f"DataFrame successfully saved to {output_filename}")

DataFrame successfully saved to sunrise_sunset_2026.xlsx


# 엑셀 다운로드

In [25]:
from google.colab import files

files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>